In [15]:
import codecs,re,math,time
import numpy as np
import itertools as it

In [2]:
with codecs.open("/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Balzac.txt","r",encoding="utf8") as input:
    texteComplet=" ".join([l.strip() for l in input.readlines()])
    

In [3]:
caracteresFiltres=re.escape(u" \n/()[]-:;,=°'_")
chunkStops=re.escape(u".?!«»")

In [4]:
texteFiltre=re.sub(ur"[%s]"%caracteresFiltres,"",texteComplet).lower()

In [5]:
chunks=re.split(ur"[%s0-9]+"%chunkStops,texteFiltre)

In [97]:
len(chunks)

29967

In [6]:
for seuil in range(50,1000,50):
    print seuil, float(sum([1 for c in chunks if len(c)<seuil]))/len(chunks)

50 0.341809323589
100 0.64841325458
150 0.83131444589
200 0.921813995395
250 0.962492074615
300 0.982147028398
350 0.991891080188
400 0.99559515467
450 0.99736376681
500 0.998598458304
550 0.999032268829
600 0.999499449394
650 0.999532819435
700 0.999566189475
750 0.999766409717
800 0.999866519838
850 0.999866519838
900 0.999933259919
950 0.99996662996


In [ ]:
class TrieNode(object):
    """
    Our trie node implementation. Very basic. but does the job
    """
    
    def __init__(self, char):
        self.char = char
        self.children = []
        # Is it the last character of the word.`
        self.word_finished = False
        # How many times this character appeared in the addition process
        self.counter = 1
    

def add(root, word):
    """
    Adding a word in the trie structure
    """
    node = root
    for char in word:
        found_in_child = False
        # Search for the character in the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found it, increase the counter by 1 to keep track that another
                # word has it as well
                child.counter += 1
                # And point the node to the child that contains this char
                node = child
                found_in_child = True
                break
        # We did not find it so add a new chlid
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            # And then point node to the new child
            node = new_node
    # Everything finished. Mark it as the end of a word.
    node.word_finished = True


def find_prefix(root, prefix):
    """
    Check and return 
      1. If the prefix exsists in any of the words we added so far
      2. If yes then how may words actually have the prefix
    """
    node = root
    # If the root node has no children, then return False.
    # Because it means we are trying to search in an empty trie
    if not root.children:
        return False, 0
    for char in prefix:
        char_not_found = True
        # Search through all the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found the char existing in the child.
                char_not_found = False
                # Assign node as the child containing the char and break
                node = child
                break
        # Return False anyway when we did not find a char.
        if char_not_found:
            return False, 0
    # Well, we are here means we have found the prefix. Return true to indicate that
    # And also the counter of the last node. This indicates how many words have this
    # prefix
    return True, node.counter

In [ ]:
righTrie=TrieNode('*')
leftTrie=TrieNode('*')
nbChunks=5000
testTexte=chunks[:nbChunks]

for chunk in testTexte:
    for pos in range(len(chunk)):
#        print chunk[pos:]
#        print     
        add(righTrie, chunk[pos:])
#        add(leftTrie, chunk[::-1])

In [84]:
print(find_prefix(righTrie, 'balza'))

(True, 68)


In [87]:
def minWidth(cuts,chaine,seuil=2):
    prevCut=0
    for c in cuts:
        if c-prevCut>seuil:
            prevCut=c
            continue
        else:
            return False
    if len(chaine)-prevCut>seuil:
        return True
    else:
        return False

def printCut(cuts,chaine):
    print chaine[0:cuts[0]]
    for n,c in enumerate(cuts[:-1]):
        print chaine[cuts[n]:cuts[n+1]]
    print chaine[cuts[-1]:]
    
def scoreCut(cuts,chaine):
    score=find_prefix(righTrie,chaine[0:cuts[0]])[1]
    for n,c in enumerate(cuts[:-1]):
#        print chaine[cuts[n]:cuts[n+1]]
        score+=find_prefix(righTrie,chaine[cuts[n]:cuts[n+1]])[1]
#    print chaine[cuts[-1]:]
    score+=find_prefix(righTrie,chaine[cuts[-1]:])[1]
    return score

In [91]:
minWidth([5,8],test)

True

In [96]:
test=testTexte[5]
nbParties=3
print test
max=0
for c in it.combinations(range(1,len(test)),nbParties):
    if minWidth(c,test):
        score=scoreCut(c,test)
        if score>max:
            print a,b, scoreCut(c,test)
            max=score
            printCut(c,test)
            print
#    print test[:a], find_prefix(righTrie, test[:a])
#    print test[a:b], find_prefix(righTrie, test[a:b])
#    print test[b:], find_prefix(righTrie, test[b:])
#    time.sleep(1)

lepetithonorénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédiehumaine
1 29 127
lep
eti
tho
norénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédiehumaine

1 29 129
lep
eti
thonorénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédie
humaine

1 29 132
lep
eti
thonorénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédiehu
maine

1 29 142
lep
eti
thonorénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédiehum
aine

1 29 205
lep
eti
thonorénefutpasunenfantprodigeilnannonçapasprématurémentquilferaitlacomédiehuma
ine

1 29 286
lep
etithonorénefutpas
une
nfantprodigeilnannonçapasprématurémentquilferaitlacomédiehumaine

1 29 306
lep
etithonorénefutpasunenf
ant
prodigeilnannonçapasprématurémentquilferaitlacomédiehumaine

1 29 575
lep
etithonorénefutpasunenfantprodigeilnannonçapasprématurém
ent
quilferaitlacomédiehumaine

1 29 715
lepetithonorénefutpasunenfantprodigeilnannonçapasprématurém
ent
qui
lferaitlacomédiehu

In [ ]:
from math import log

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = open("words-by-frequency.txt").read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))